In [5]:
def extract_final_response(response):
    """response 객체의 messages에서 마지막 AIMessage의 content를 추출"""
    messages = response.get('messages', [])
    for message in reversed(messages):
        if hasattr(message, "content") and isinstance(message.content, str) and message.content.strip():
            return message.content.strip()
    return "[No final response found]"


In [6]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.prebuilt import create_react_agent

from langchain_openai import ChatOpenAI
import asyncio
import nest_asyncio  # 이 두개는 jupyter lab에서만
nest_asyncio.apply() # 설정하는 함수입니다. py에선 사용안합니다.
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini")

async def main():
    async with MultiServerMCPClient(
        {
        "math": {
            "command": "python",
            "args": ["./server/math_server.py"],
            "transport": "stdio",
        },
        "weather": {
            "command": "python",
            "args": ["./server/weather_server.py"],
            "transport": "stdio",
        },			
        "PDF_FAISS": {
            "command": "python",
            "args": ["./server/faiss_server.py"],
            "transport": "stdio",
        },

        }
    ) as client:
        agent = create_react_agent(model, client.get_tools())
        math_response = await agent.ainvoke({"messages": "what's (3 + 5) x 12?"})
        weather_response = await agent.ainvoke({"messages": "what is the weather in nyc?"})
        faiss_response = await agent.ainvoke({"messages": "langgraph에서 adaptive rag를 구축하는 방법을 알려주세요."})
        # 각 응답에서 핵심 응답 출력
        print("📘 Math Response:")
        print(extract_final_response(math_response))
        print("\n🌤️ Weather Response:")
        print(extract_final_response(weather_response))
        print("\n🧠 FAISS / LangGraph Response:")
        print(extract_final_response(faiss_response))
# asyncio 이벤트 루프를 통해 main() 실행
asyncio.run(main())

📘 Math Response:
The result of \((3 + 5) \times 12\) is 96.

🌤️ Weather Response:
The weather in New York City is always sunny!

🧠 FAISS / LangGraph Response:
Adaptive RAG (Retrieval-Augmented Generation) in LangGraph can be constructed by combining query analysis and an active/self-corrective approach to enhance the retrieval results based on user queries. Here's a step-by-step guide to implementing Adaptive RAG in LangGraph:

### 1. Set Up Your Environment
First, you need to install the necessary packages and set up your API keys:

```python
!pip install -U langchain_community tiktoken langchain-openai langchain-cohere langchainhub chromadb

import getpass
import os

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("TAVILY_API_KEY")
```

### 2. Create an Index
Start by creating an index for your documents. This is essential for the retrieval system.

```python
from langchain.text_splitte